# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items

In [1]:
name = "ItemCFResid";

In [2]:
using NBInclude
@nbinclude("ItemCollaborativeFilteringBase.ipynb");

## Compute cosine correlations

In [3]:
function get_correlation_matrix_outdir(residual_alphas)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $residual_alphas"
    training = get_residuals("training", residual_alphas)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_alphas = ["ItemCF.$K" for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])]
item_cf_params = [[
        cf_params(
            name = "ItemCFResid.$K",
            training_residuals = ["UserItemBiases"; item_alphas],
            validation_residuals = ["UserItemBiases"; item_alphas],
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(["UserItemBiases"; item_alphas]),
            K = K,
            λ = [1., 1., 0.],
        ) for K in downcast_to_int.([2^8])
    ];
];

## Train models

In [ ]:
for param in item_cf_params
    optimize_model(param)
end

Progress: 100%|███████████████████████████| Time: 0:03:39 (55.30 ms/it)
[ Debug: 20211231 14:44:51 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2561244923782138,-0.00500613411669654,0.0007931270595019888,-7.081951990093505e-5) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2012661970345693,2.858771488249437,-0.6184841084904319,0.05904739184038273): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDa

Iter     Function value   Gradient norm 
     0     1.256124e+00     5.006134e-03
 * Current step size: 1.0
 * time: 0.022655010223388672
 * g(x): [-0.00500613411669654, 0.0007931270595019888, -7.081951990093505e-5]
 * x: [0.1, 0.9, 11.443905449241216]


Progress: 100%|███████████████████████████| Time: 0:03:38 (54.87 ms/it)
[ Debug: 20211231 14:48:36 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2560991864514852,-0.004854938345107371,0.0007633597868912047,-6.915344722977888e-5) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2161823890348296,2.902052844225679,-0.6234861037754464,0.060057626464899504): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, Ratings

     1     1.255744e+00     1.219119e-04
 * Current step size: 27.786448094414588
 * time: 1089.0223190784454
 * g(x): [-0.00010840053744305696, -0.00012191193170185788, -2.693972180505159e-5]
 * x: [0.23910268578726646, 0.8779618161288724, 11.445873272155014]


Progress: 100%|███████████████████████████| Time: 0:03:12 (48.32 ms/it)
[ Debug: 20211231 15:06:18 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2557436267124915,3.273722999074925e-6,-7.649809594825173e-5,-2.604615406580052e-5) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.7207405629179902,4.351626546318109,-0.7845144900117014,0.09644581000202314): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDa

...

Progress: 100%|███████████████████████████| Time: 0:03:49 (57.65 ms/it)
[ Debug: 20211231 16:26:17 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.2556418595336216,4.844741610142694e-7,3.5327656764912834e-7,5.94570903869958e-9) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(2.3868798188657223,6.919922331299344,-1.0545864236169569,0.06940549530521468): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsData

     8     1.255642e+00     2.277294e-07
 * Current step size: 0.8905409876179644
 * time: 6313.7614760398865
 * g(x): [2.277293708036575e-7, -1.834167610184601e-7, 1.2737367689518493e-9]
 * x: [0.17949008219636026, 0.8723256369460205, 23.947899457325477]


Progress: 100%|███████████████████████████| Time: 0:04:31 (68.43 ms/it)
[ Debug: 20211231 16:34:45 loss: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(1.255641859510327,-7.078817758538532e-9,4.369208227196094e-9,-2.8662635676562452e-11) β: Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}}(2.38644990133269,6.918266938453817,-1.054338038118929,0.0694100821400346): λ ForwardDiff.Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDataset}, Float64}, Float64, 3}[Dual{ForwardDiff.Tag{var"#validation_mse#17"{var"#14#16"{cf_params, Dict{String, typeof(get_abs_neighborhood)}, Int64, Matrix{Float64}}, RatingsDataset, RatingsDatas

     9     1.255642e+00     1.439361e-09
 * Current step size: 0.9759831704135143
 * time: 6864.404228925705
 * g(x): [-1.439360998211004e-9, -1.408056525475718e-10, 2.619259759706045e-12]
 * x: [0.17948731395513118, 0.8723354575709694, 23.946905176074917]


Progress: 100%|███████████████████████████| Time: 0:38:02 ( 0.54  s/it)
[ Info: 20211231 17:19:14 training set: RMSE 1.0772049186715105 MAE 0.779596724209766 R2 0.06668462869001401
[ Info: 20211231 17:19:21 validation set: RMSE 1.1205542644202133 MAE 0.81166477002724 R2 0.013576840927245537
